# Pilot Ray Example

In [ ]:
# Pilot-Ray
import os, sys
sys.path.insert(0, "../../..")
import pilot.streaming
import ray

#configure loggin
import logging
logging.getLogger().setLevel(logging.DEBUG)
logging.getLogger("stevedore.extension").setLevel(logging.CRITICAL)
logging.getLogger("keystoneauth").setLevel(logging.CRITICAL)
logging.getLogger("urllib3.connectionpool").setLevel(logging.CRITICAL)
logging.getLogger("asyncio").setLevel(logging.CRITICAL)

sys.modules['pilot.streaming']

RESOURCE_URL_HPC="slurm+ssh://login03.chn"
RESOURCE_URL_LOCAL="ssh://localhost"
WORKING_DIRECTORY=os.path.join(os.environ["HOME"], "work")


## Setup Cluster

In [ ]:
pilot_compute_description = {
    "resource":RESOURCE_URL_HPC,
    "working_directory": os.path.join(WORKING_DIRECTORY, "work"),
    "cores_per_node": 1,
    "gpus_per_node": 0,    
    "queue": "normal",
    "walltime": 59,
    "type":"ray"
}

In [ ]:
ray_pilot = pilot.streaming.PilotComputeService.create_pilot(pilot_compute_description)
ray_pilot.wait()

## Ray Test

In [ ]:
#ray_client = ray.init(address="ray://128.55.64.12:10001")
ray_client = ray_pilot.get_context()

In [ ]:
#ray_client = ray.init(address="128.55.64.12:6379", ignore_reinit_error=True)

In [ ]:
ray.cluster_resources()

In [ ]:
# create a remote function
@ray.remote
def f(x):
    return x * x

# call the remote function ten times in parallel
# and return the results as a list
ray.get([f.remote(i) for i in range(10)])